In [2]:
!pip install Qiskit ipywidgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.5 MB 634 kB/s 
     |████████████████████████████████| 18.0 MB 9.3 MB/s 
     |████████████████████████████████| 240 kB 42.3 MB/s 
     |████████████████████████████████| 198 kB 52.9 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 112 kB 49.3 MB/s 
     |████████████████████████████████| 1.6 MB 36.9 MB/s 
     |████████████████████████████████| 943 kB 34.8 MB/s 
     |████████████████████████████████| 37.5 MB 1.2 MB/s 
     |████████████████████████████████| 49 kB 7.0 MB/s 
     |████████████████████████████████| 49 kB 6.6 MB/s 
     |████████████████████████████████| 38.1 MB 373 kB/s 
     |████████████████████████████████| 4.0 MB 42.2 MB/s 
     |████████████████████████████████| 112 kB 50.5 MB/s 
  Created wheel for Qiskit: filename=qiskit-0.36.2-py3-none-any.whl size=11933 sha256=f44d71c485

In [3]:
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install qiskit_optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 154 kB 647 kB/s 
     |████████████████████████████████| 610 kB 4.7 MB/s 
  Created wheel for docplex: filename=docplex-2.23.222-py3-none-any.whl size=662847 sha256=6144c9c224535e7df65292ae416d2380eb67651386c1917165e4ba4878825f6e
  Stored in directory: /root/.cache/pip/wheels/a7/c9/fb/cee5a89f304e77a39c466e625ac2830434b76eb8384999d116
Successfully built docplex


In [5]:
!pip install qiskit-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install cplex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43.3 MB 1.6 MB/s 


In [7]:
!pip install ibm-quantum-widgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 993 kB 648 kB/s 


In [8]:
!pip install docplex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
#provider = IBMQ.load_account()

In [10]:
import time
from typing import List, Optional, Any
import numpy as np
import matplotlib.pyplot as plt

from docplex.mp.model import Model

from qiskit import BasicAer
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_optimization.algorithms import CobylaOptimizer, MinimumEigenOptimizer
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.algorithms.admm_optimizer import ADMMParameters, ADMMOptimizer
from qiskit_optimization.translators import from_docplex_mp

from qiskit.utils.algorithm_globals import algorithm_globals
algorithm_globals.massive=True

# If CPLEX is installed, you can uncomment this line to import the CplexOptimizer.
# CPLEX can be used in this tutorial to solve the convex continuous problem,
# but also as a reference to solve the QUBO, or even the full problem.
#
from qiskit_optimization.algorithms import CplexOptimizer

In [11]:
# define COBYLA optimizer to handle convex continuous problems.
cobyla = CobylaOptimizer()

# define QAOA via the minimum eigen optimizer
qaoa = MinimumEigenOptimizer(QAOA(quantum_instance=BasicAer.get_backend("statevector_simulator")))

# exact QUBO solver as classical benchmark
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())  # to solve QUBOs

# in case CPLEX is installed it can also be used for the convex problems, the QUBO,
# or as a benchmark for the full problem.
#
# cplex = CplexOptimizer()

In [12]:
# construct model using docplex
mdl = Model("ex7")

#3 servers and 5 virtual machines
#all servers have the same idle and dynamic power consumption
#all servers have the same maximum cpu capacity

s0 = mdl.binary_var(name="s0")
s1 = mdl.binary_var(name="s1")
s2 = mdl.binary_var(name="s2")

v00 = mdl.binary_var(name="v00")
v10 = mdl.binary_var(name="v10")
v20 = mdl.binary_var(name="v20")
v01 = mdl.binary_var(name="v01")
v11 = mdl.binary_var(name="v11")
v21 = mdl.binary_var(name="v21")
v02 = mdl.binary_var(name="v02")
v12 = mdl.binary_var(name="v12")
v22 = mdl.binary_var(name="v22")

#pi = mdl.continuous_var(name="pi")
#pd = mdl.continuous_var(name="pd")
pi = 10
pd = 5

c = 100

u00 = mdl.integer_var(name="u00")
u10 = mdl.integer_var(name="u10")
u20 = mdl.integer_var(name="u20")
u01 = mdl.integer_var(name="u01")
u11 = mdl.integer_var(name="u11")
u21 = mdl.integer_var(name="u21")
u02 = mdl.integer_var(name="u02")
u12 = mdl.integer_var(name="u12")
u22 = mdl.integer_var(name="u22")

In [13]:
sum0 = (u00*v00+u10*v10+u20*v20)
sum1 = (u01*v01+u11*v11+u21*v21)
sum2 = (u02*v02+u12*v12+u22*v22)

In [14]:

mdl.maximize((pi*s0+pd*(sum0))+(pi*s1+pd*(sum1))+(pi*s2+pd*(sum2)))
#mdl.add_constraint(v + 2 * w + t + u <= 3, "cons1")
#mdl.add_constraint(v + w + t >= 1, "cons2")
#mdl.add_constraint(v + w == 1, "cons3")

# load quadratic program from docplex model
qp = from_docplex_mp(mdl)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: ex7

Maximize
 obj: 10 s0 + 10 s1 + 10 s2 + [ 10 v00*u00 + 10 v10*u10 + 10 v20*u20
      + 10 v01*u01 + 10 v11*u11 + 10 v21*u21 + 10 v02*u02 + 10 v12*u12
      + 10 v22*u22 ]/2
Subject To

Bounds
 0 <= s0 <= 1
 0 <= s1 <= 1
 0 <= s2 <= 1
 0 <= v00 <= 1
 0 <= v10 <= 1
 0 <= v20 <= 1
 0 <= v01 <= 1
 0 <= v11 <= 1
 0 <= v21 <= 1
 0 <= v02 <= 1
 0 <= v12 <= 1
 0 <= v22 <= 1

Binaries
 s0 s1 s2 v00 v10 v20 v01 v11 v21 v02 v12 v22

Generals
 u00 u10 u20 u01 u11 u21 u02 u12 u22
End



In [15]:
admm_params = ADMMParameters(
                            rho_initial=1001,
                            beta=1000,
                            factor_c=900,
                            maxiter=100,
                            three_block=True, tol=1.e-6
                        )

In [16]:
# define QUBO optimizer
qubo_optimizer = exact
#qubo_optimizer = cplex  # uncomment to use CPLEX instead

# define classical optimizer
convex_optimizer = cobyla
# convex_optimizer = cplex  # uncomment to use CPLEX instead

# initialize ADMM with classical QUBO and convex optimizer
admm = ADMMOptimizer(params=admm_params,
                     qubo_optimizer=qubo_optimizer,
                     continuous_optimizer=convex_optimizer,
                    )

In [17]:

 
#QiskitAlgorithmGlobals.massive = True;

In [ ]:
# run ADMM to solve problem
%tb
#algorithm_globals.massive=True
#algorithm_globals = (massive=True)
#admm.OperatorBase._check_massive(massive=True)
result = admm.solve(qp)

NameError: ignored

In [ ]:
print("x={}".format(result.x))
print("fval={:.2f}".format(result.fval))

In [ ]:
plt.plot(result_q.state.residuals)
plt.xlabel("Iterations")
plt.ylabel("Residuals")
plt.show()

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright